In [1]:
%%bash
source /data/reddylab/software/miniconda2/bin/activate alex
module load gcc
python /data/reddylab/Alex/reddylab_utils/scripts/featurecounts_output_to_df.py \
    -featurecounts /data/reddylab/Revathy/collabs/atac-seq_Siklenka/analysis/Th0_Th17.72hrs.UnionPeakSet.featurecounts.txt \
    -outfiles /data/reddylab/Revathy/collabs/atac-seq_Siklenka/analysis/Th0_Th17.72hrs.UnionPeakSet.featurecounts.df.txt \
    --drop-suffixes .masked.dups_marked.bam \
    --join-input-files \


In [2]:
%%bash
source /data/reddylab/software/miniconda2/bin/activate alex
module load gcc
python /data/reddylab/Alex/reddylab_utils/scripts/featurecounts_output_to_df.py \
    -featurecounts /data/reddylab/Revathy/dev/atac-seq_Siklenka/analysis/Th0_Th17_new.72hrs.UnionPeakSet.featurecounts.txt \
    -outfiles /data/reddylab/Revathy/dev/atac-seq_Siklenka/analysis/Th0_Th17_new.72hrs.UnionPeakSet.featurecounts.combined.df.txt \
    --drop-suffixes masked.dups_marked.bam masked.dups_marked.nsorted.bam \

In [2]:
%%bash
cd /data/reddylab/Revathy/collabs/atac-seq_Siklenka/analysis
file=Th0_Th17.72hrs.UnionPeakSet.featurecounts
cat ${file}.df.txt | cut -f1,3,4,5,6,7,8 | sed '1{ s/KS133.Th0.72hrs.ASTARR.insert.rep1/Th0.72hrs.ASTARR.insert.rep1/; s/KS133.Th0.72hrs.ASTARR.insert.rep2/Th0.72hrs.ASTARR.insert.rep2/; s/KS134.Th17.72hrs.ASTARR.insert.rep1/Th17.72hrs.ASTARR.insert.rep1/; s/KS134.Th17.72hrs.ASTARR.insert.rep2/Th17.72hrs.ASTARR.insert.rep2/; s/KS136.Th17.72hrs.ASTARR.insert.rep3/Th17.72hrs.ASTARR.insert.rep3/; s/KS136.Th17.72hrs.ASTARR.insert.rep4/Th17.72hrs.ASTARR.insert.rep4/; s/^/#/; }' > ${file}.txt 


In [4]:
!head /data/reddylab/Revathy/collabs/atac-seq_Siklenka/analysis/Th0_Th17.72hrs.UnionPeakSet.featurecounts.txt

#Geneid	Th0.72hrs.ASTARR.insert.rep2	Th17.72hrs.ASTARR.insert.rep1	Th17.72hrs.ASTARR.insert.rep2	Th17.72hrs.ASTARR.insert.rep3	Th17.72hrs.ASTARR.insert.rep4
chr1_4491842_4492115	18	10	13	28	19
chr1_4492489_4493388	53	46	36	40	48
chr1_4496438_4497087	58	45	58	59	53
chr1_4571640_4571997	25	32	16	29	34
chr1_4600559_4601117	44	308	332	439	369
chr1_4671564_4672001	141	73	92	104	87
chr1_4724647_4724940	24	85	60	63	55
chr1_4748181_4748505	40	24	28	65	51
chr1_4774994_4775958	93	166	157	121	132


Copy read_count.mapped files to new folder for ease of library size calculation

In [3]:
%%bash
mkdir -p /data/reddylab/Revathy/dev/atac-seq_Siklenka/data/atac_seq/readCount

cp /data/reddylab/Revathy/dev/atac-seq_Siklenka/processing/atac_seq/Siklenka_6420_200804A5-pe-blacklist-removal/*.Th17.*.mapped /data/reddylab/Revathy/dev/atac-seq_Siklenka/data/atac_seq/readCount
cp /data/reddylab/Revathy/dev/atac-seq_Siklenka/processing/atac_seq/Siklenka_6420_200805A5-pe-blacklist-removal/*.Th0.*.mapped /data/reddylab/Revathy/dev/atac-seq_Siklenka/data/atac_seq/readCount


Calculate library sizes for Th0 and Th17

In [5]:
%%bash
# Generate lib_sizes for each ATAC-seq library (number of mapped_reads)
cd /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/readCount
echo -n > /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes.txt
for ii in *.{Th0,Th17}.*.bowtie.log.read_count.mapped;
do
    iname=$(basename ${ii} | sed 's@.bowtie.log.read_count.mapped@@')
    echo -e $iname"\t"$(cut -f2 ${ii}) \
        >> /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes.tmp.txt
done

In [6]:
%%writefile /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes.names.txt
KS133.Th0.72hrs.ASTARR.insert.rep1
KS133.Th0.72hrs.ASTARR.insert.rep2
KS134.Th17.72hrs.ASTARR.insert.rep1
KS134.Th17.72hrs.ASTARR.insert.rep2
KS136.Th17.72hrs.ASTARR.insert.rep3
KS136.Th17.72hrs.ASTARR.insert.rep4

Overwriting /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes.names.txt


In [7]:
%%bash
paste /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes.names.txt /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes.tmp.txt \
    | cut -f1,3 \
    > /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes.txt

In [8]:
%%bash
cut -d "." -f2,3,4,5,6 /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes.txt > /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes_format.txt


In [12]:
%%writefile /data/reddylab/Revathy/scripts/atacseq.deseq2.R
#!/usr/bin/env Rscript
library(DESeq2)
library(ggplot2)
library(matrixStats)
library(dplyr)

suppressPackageStartupMessages(library("argparse"))

# Rewrite DESeq2 plotPCA function to specify which PC to plot
plotPCA_ = function(object, intgroup="condition", ntop=500, returnData=FALSE, pcX=1, pcY=2, ignoreReps=FALSE)
{
  # calculate the variance for each gene
  rv <- rowVars(assay(object))

  # select the ntop genes by variance
  select <- order(rv, decreasing=TRUE)[seq_len(min(ntop, length(rv)))]

  # perform a PCA on the data in assay(x) for the selected genes
  pca <- prcomp(t(assay(object)[select,]))

  # the contribution to the total variance for each component
  percentVar <- pca$sdev^2 / sum( pca$sdev^2 )

  if (!all(intgroup %in% names(colData(object)))) {
    stop("the argument 'intgroup' should specify columns of colData(dds)")
  }

  intgroup.df <- as.data.frame(colData(object)[, intgroup, drop=FALSE])
  
  # add the intgroup factors together to create a new grouping factor
  group <- if (length(intgroup) > 1) {
    factor(apply( intgroup.df, 1, paste, collapse=":"))
  } else {
    colData(object)[[intgroup]]
  }
    
  if (returnData) {
    attr(d, "percentVar") <- c(percentVar[pcX], percentVar[pcY])
    return(d)
  }

  if (!ignoreReps){
      # Add replicate info
      rep <- colData(object)[['rep']]

        # assembly the data for the plot
      d <- data.frame(PCX=pca$x[,pcX], PCY=pca$x[,pcY], group=group, rep=rep, intgroup.df, name=colnames(object))

      ggplot(data=d, aes_string(x="PCX", y="PCY", color="group", shape="rep")) + geom_point(size=3) + 
        xlab(paste0("PC", pcX, ": ",round(percentVar[pcX] * 100),"% variance")) +
          ylab(paste0("PC", pcY, ": ",round(percentVar[pcY] * 100),"% variance"))       
  } else {
#       sex <- colData(object)[['sex']]

        # assembly the data for the plot
      d <- data.frame(PCX=pca$x[,pcX], PCY=pca$x[,pcY], group=group, intgroup.df, name=colnames(object)) # , sex=sex
#     , shape="sex"
      ggplot(data=d, aes_string(x="PCX", y="PCY", color="group")) + geom_point(size=3) + 
        xlab(paste0("PC", pcX, ": ",round(percentVar[pcX] * 100),"% variance")) +
          ylab(paste0("PC", pcY, ": ",round(percentVar[pcY] * 100),"% variance"))       
      
  }
#     + coord_fixed()
}

# create parser object
parser <- ArgumentParser()

# specify our desired options 
# by default ArgumentParser will add an help option 
parser$add_argument("-c", "--controls", nargs="+", help="Featurecounts file for controls")
parser$add_argument("-t", "--treatments", nargs="+", help="Featurecounts file for treatments")
parser$add_argument("--counts", nargs="+", help="Count matrix (or matrices). Count columns should match controls and treatments specified")
parser$add_argument("--outdir", help="Output directory where result files will be saved.")
parser$add_argument("--lfcShrink", required=F, 
                    help="Perform effect size shrinkage (types: normal, apeglm and ashr. See Zhu, Ibrahim, and Love 2018).")
parser$add_argument("--lib-sizes", required=F, 
                    help="Use library sizes as scaling factor overwritting the default total counts per library")
parser$add_argument("--counts-skip-ncols", required=F, 
                    help="Skip n first columns from count matrix")
parser$add_argument("--counts-only-treatments-and-controls", required=F, action="store_true", default=FALSE,
                    help="Keep only the counts from the treatment and control samples (affects dispersion estimates, but can speed up things significantly)")
parser$add_argument("--no-filter", required=F, action="store_true", default=FALSE,
                    help="By default, lowly expressed regions in only 1 sample are removed. Specify if no filter should be done")
parser$add_argument("--no-model-reps", required=F, action="store_true", default=FALSE,
                    help="Ignore replicates when plotting PCA components")
parser$add_argument("--additional-metadata", required=F, help="For genotype and sex info")

# get command line options, if help option encountered print help and exit,
# otherwise if options not found on command line then set defaults, 
args <- parser$parse_args()
treatments <- args$treatments
controls <- args$controls
outdir <- args$outdir

ff <- args$counts
dd <- lapply(ff, read.csv, header=TRUE, sep="\t", stringsAsFactors=FALSE, row.names=1)
xx <- do.call(cbind, dd)

countData <- xx
if (!is.null(args$counts_skip_ncols)){
    countData <- xx[, -c(1:args$counts_skip_ncols)]
}

colnames(countData)[colnames(countData)%in%append(treatments, controls)]
if (args$counts_only_treatments_and_controls){
#     countData <- xx %>% select(unlist(append(treatments, controls)))
    countData <- subset(xx, select = append(treatments, controls))
}

controls
treatments

samples_no_reps <- gsub("(.*)\\..*", "\\1", colnames(countData))
reps <- gsub(".*\\.(.*)", "\\1", colnames(countData))

conditions <- samples_no_reps
conditions

if (!args$no_model_reps){
    if (!is.null(args$additional_metadata)){
        sex_genotype <- read.table(args$additional_metadata, sep="\t", header=1, row.names=1)
        sex <- sex_genotype[colnames(countData), c("Sex")]
#         genotypes <- sex_genotype[colnames(countData), c("Genotype")]

        colData <- data.frame(condition=conditions, rep=reps, sex=sex, row.names=colnames(countData))
        dds <- DESeqDataSetFromMatrix(countData = countData,
                                      colData = colData,
                                      design = ~ rep + sex + condition)
        
    } else {
        colData <- data.frame(condition=conditions, rep=reps, row.names=colnames(countData))
        dds <- DESeqDataSetFromMatrix(countData = countData,
                                      colData = colData,
                                      design = ~ rep + condition)
    }
} else {
    
    if (!is.null(args$additional_metadata)){
        sex_genotype <- read.table(args$additional_metadata, sep="\t", header=1, row.names=1)
        sex <- sex_genotype[colnames(countData), c("Sex")]

        colData <- data.frame(condition=conditions, sex=sex, row.names=colnames(countData))
        dds <- DESeqDataSetFromMatrix(countData = countData,
                                      colData = colData,
                                      design = ~ sex + condition)
        
    } else {
        colData <- data.frame(condition=conditions, row.names=colnames(countData))
        dds <- DESeqDataSetFromMatrix(countData = countData,
                                      colData = colData,
                                      design = ~ condition)
        
    }
}

# Optionally: if lib_sizes provided, manually assigned scale factors
if (!is.null(args$lib_sizes)){
    lib_sizes <- read.table(args$lib_sizes, sep="\t", row.names=1)
    lib_sizes <- lib_sizes[colnames(countData), ]
    lib_sizes_scale_factors <- lib_sizes/min(lib_sizes)
    sizeFactors(dds) <- lib_sizes_scale_factors    
}


# Reorder condition comparisons to always compute log2 fold changes of treat vs control
conditions_pair <- unique(conditions)
control_conditions <- unique(gsub("(.*)\\..*", "\\1", controls))
treatment_conditions <- unique(gsub("(.*)\\..*", "\\1", treatments))

# Relevel to setup control as reference
dds$condition <- relevel(dds$condition, ref = control_conditions)

# Pre-filtering low count genes
# dds <- dds[ rowSums(counts(dds)) > 5, ]
if (!args$no_filter){
    dds <- dds[ rowSums(fpm(dds, robust = FALSE)>=2) > 1, ]
}

# Run DESeq2
dds <- DESeq(dds)

# rlog transformed values (typically used for clustering - see DESeq2 manual for details)
rld <- rlog(dds)
nrow(results(dds))
head(assay(rld))

# Calculate comparison string name 
treat_vs_contr <- paste0(treatment_conditions, '_vs_', control_conditions)

# Extracting transformed values for PCA analysis
vsd <- vst(dds, blind=FALSE)

# Create PCA plot for the first and second PCs
pdf(paste0(outdir, '/', treat_vs_contr, '.pca.1_vs_2.pdf'), width=8, height=5)
plotPCA_(vsd, intgroup=c('condition'), ntop=500, pcX=1, pcY=2, ignoreReps=args$no_model_reps)
dev.off()
# Create PCA plot for the second and third PCs
pdf(paste0(outdir, '/', treat_vs_contr, '.pca.2_vs_3.pdf'), width=8, height=5)
plotPCA_(vsd, intgroup=c('condition'), ntop=500, pcX=2, pcY=3, ignoreReps=args$no_model_reps)
dev.off()
# Create PCA plot for the third and fourth PCs
pdf(paste0(outdir, '/', treat_vs_contr, '.pca.3_vs_4.pdf'), width=8, height=5)
plotPCA_(vsd, intgroup=c('condition'), ntop=500, pcX=3, pcY=4, ignoreReps=args$no_model_reps)
dev.off()


c('condition',  
  conditions_pair[conditions_pair %in% treatment_conditions], 
  conditions_pair[conditions_pair %in% control_conditions])
res <- results(dds, 
               contrast=c('condition', 
                          conditions_pair[conditions_pair %in% treatment_conditions],
                          conditions_pair[conditions_pair %in% control_conditions]), 
               alpha=0.05,
               independentFiltering=TRUE)  # TODO: Test how the results look without independent filtering

# Print out result names
resultsNames(dds)


# Shrink log2 fold-change values?
if (!is.null(args$lfcShrink)){
    targ_contrast <- paste0("condition_", treat_vs_contr)
    res <- lfcShrink(dds, 
                     coef=targ_contrast, 
                     type=args$lfcShrink)
}

# Sort by adjusted p-value (FDR)
(resOrdered <- res[order(res$padj),])


# Print out summary of results
summary(resOrdered, alpha=0.05)

# Try IHW multiple hypothesis testing package
library("IHW")
resIHW <- results(dds, 
                  alpha=0.05,
                  contrast=c('condition', 
                          conditions_pair[conditions_pair %in% treatment_conditions],
                          conditions_pair[conditions_pair %in% control_conditions]),
                  filterFun=ihw
                 )
summary(resIHW, alpha=0.05)


# Create plot for the log2 fold changes from the treatment over the mean of normalized counts
pdf(paste0(outdir, '/', treat_vs_contr, '.pdf'))
plotMA(res, main=paste0("DESeq2_", treat_vs_contr), ylim=c(-10,10))
dev.off()

# Create plot for dispersion of normalized counts
pdf(paste0(outdir, '/', treat_vs_contr, '.mean_counts_dispersion.pdf'))
plotDispEsts(dds)
dev.off()

# To check if there are too many outliers, plot the Cook distances 
pdf(paste0(outdir, '/', treat_vs_contr, '.cook_distances.pdf'))
par(mar=c(8,5,2,2))
boxplot(log10(assays(dds)[["cooks"]]), range=0, las=2)
dev.off()

# Save sorted results in a text file
write.table(resOrdered, 
            file=paste0(outdir, '/', treat_vs_contr, '.txt'),
            quote = FALSE, row.names=TRUE, sep = '\t')

# Save rlog transformed values
write.table(assay(rld), 
            file=paste0(outdir, '/', treat_vs_contr, '.rlog.txt'),
            quote = FALSE, row.names=TRUE, sep = '\t')



Overwriting /data/reddylab/Revathy/scripts/atacseq.deseq2.R


In [13]:
%%bash
mkdir -p /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/tmp
source /data/reddylab/software/miniconda2/bin/activate alex_dev
module load gcc
sbatch -p all \
    --mem 16G \
    -o /data/reddylab/Revathy/collabs/atac-seq_Siklenka/logs/tmp_atac_deseq.out \
    <<'EOF'
#!/bin/bash
COMPARISONS=( \
    Th17.72hrs.ASTARR.insert-Th0.72hrs.ASTARR.insert \
)

COMPARISON=${COMPARISONS[${SLURM_ARRAY_TASK_ID}]}
TREAT=$(echo $COMPARISON | cut -d"-" -f1)
CONTROL=$(echo $COMPARISON | cut -d"-" -f2)
FACTOR=$(echo ${CONTROL} | cut -d. -f1)

Rscript /data/reddylab/Revathy/scripts/atacseq.deseq2.R \
    --lfcShrink ashr \
    --no-model-reps \
    --treatments $(/bin/grep ${TREAT} /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes_format.txt | cut -f1) \
    --controls $(/bin/grep ${CONTROL} /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes_format.txt | cut -f1) \
    --lib-sizes /data/reddylab/Revathy/collabs/atac-seq_Siklenka/data/atac_seq/counts/lib_sizes_format.txt \
    --outdir /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/tmp/ \
    --counts /data/reddylab/Revathy/collabs/atac-seq_Siklenka/analysis/Th0_Th17.72hrs.UnionPeakSet.featurecounts.txt
EOF

Submitted batch job 25793273


In [3]:
!wc -l /data/reddylab/Revathy/collabs/atac-seq_Siklenka/analysis/Th0_Th17.72hrs.UnionPeakSet.featurecounts.txt

84223 /data/reddylab/Revathy/collabs/atac-seq_Siklenka/analysis/Th0_Th17.72hrs.UnionPeakSet.featurecounts.txt


In [1]:
!head /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/tmp/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.txt

baseMean	log2FoldChange	lfcSE	pvalue	padj
chr4_143288226_143289757	1647.73336636732	3.81726459784635	0.149421823937938	2.26509166736847e-145	1.54875642756319e-140
chr16_25010772_25012159	1485.52387348661	3.48472723761304	0.140449160020006	4.20040679819601e-137	1.43601407413326e-132
chr2_181099140_181100692	1928.35072840258	2.9614912246334	0.12138931489356	2.19773839527376e-132	5.00901209256145e-128
chr17_24922152_24923413	1302.95899109185	4.05652786705213	0.173908025374107	6.24351466900603e-122	1.06725078873322e-117
chr6_88827500_88829063	1481.49179677977	3.53133982990557	0.152986375873328	3.99629865460984e-119	5.46493841017895e-115
chr1_36965264_36966375	1868.2189488516	2.32734236110937	0.102147391471507	1.24159084180454e-115	1.41489623013975e-111
chr12_70890543_70891870	1399.96979228548	3.98211754387464	0.178451046369452	7.65614613393013e-112	7.47841417010676e-108
chr1_93529464_93530987	1365.00075899984	3.19374550045465	0.143893638020533	3.44938808156298e-110	2.94814887596086e-106
ch

In [12]:
%%bash
FDR=0.05
COMPARISONS=( \
    Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert \
)
cd /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2
for COMP in ${COMPARISONS[@]};
do
    # Save expressed genes (more than 5 normalized counts)
    tail -n+2 ${COMP}.txt |  cut -f1 > ${COMP}.accessible_peaks.txt 
    
    # Save significant upreg genes
    tail -n+2 ${COMP}.txt |  awk -vFDR=${FDR} '{if(($NF<FDR) && ($3>0)){print $1}}' |  cut -d. -f1 > ${COMP}.increase.txt 

    # Save significant downreg genes
    tail -n+2 ${COMP}.txt |  awk -vFDR=${FDR} '{if(($NF<FDR) && ($3<0)){print $1}}' |  cut -d. -f1 > ${COMP}.decrease.txt 

done


In [13]:
%%bash
cd /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2
COMPARISONS=( \
    Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert \
)
for ENRICH in increase decrease;
do
    for ii in ${COMPARISONS[@]};
    do
        tail -n+2 ${ii}.${ENRICH}.txt | cut -f1 | sed 's@_@\t@g' > ${ii}.${ENRICH}.bed 
    done
done

In [14]:
%%bash
# For all, just take one of the rlog files
cd /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2
COMPARISONS=( \
    Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert \
)
for ii in ${COMPARISONS[@]};
do
    tail -n+2 ${ii}.rlog.txt | cut -f1 | sed 's@_@\t@g' > ${ii}.rlog.bed 
done


In [15]:
%%bash 
wc -l /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.accessible_peaks.txt
wc -l /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.increase.txt
wc -l /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.decrease.txt



68375 /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.accessible_peaks.txt
21583 /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.increase.txt
11740 /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.decrease.txt


Add closest gene info

In [16]:
%%bash
cd /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/
sbatch -p all \
    --cpus-per-task 1 \
    --mem 8G \
    -o /data/reddylab/Revathy/collabs/atac-seq_Siklenka/logs/atacseq_deseq2.add_closest_gene.rlog.out \
    <<'EOF'
#!/bin/bash
SAMPLE="Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.rlog.txt"

tail -n+2 ${SAMPLE} \
| cut -f1 \
| awk -F_ -vOFS="\t" '{print $1, $2, $3, $1"_"$2"_"$3}' \
| sort -k1,1 -k2,2n \
| bedtools closest \
    -nonamecheck \
    -D a \
    -a - \
    -b /data/reddylab/Reference_Data/Gencode/vM19/gencode.vM19.basic.annotation.bed \
| awk -vOFS="\t" \
    '{if(last==$4){genes=$11","genes;}else{if(genes){print peak, genes, distance}peak=$4; genes=$11; distance=$13; last=peak}}END{if(last==$4){genes=$11","genes;print peak, genes, distance}}' \
> ${SAMPLE/.txt/.closest_gene.txt}


EOF

Submitted batch job 25774122


In [17]:
import pandas as pd
samples = ["Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.rlog"] 
data_dir = '/data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2'
for sample in samples:
    df = pd.read_csv("%s/%s.txt" % (data_dir, sample), sep='\t')
    df_closest= pd.read_csv("%s/%s.closest_gene.txt" % (data_dir, sample), index_col=0, sep='\t',
                            names=['peak', 'genes', 'distance'])
    df.join(df_closest, how='outer')\
        .to_csv("%s/%s.with_closest_gene.txt" % (data_dir, sample), sep='\t')

In [18]:
%%bash
module load bedtools2
tail -n+2 /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.txt | awk -vOFS="\t" '$NF<0.05{split($1, coords, "_"); print coords[1], coords[2], coords[3], $1, $3, "."}' \
    | sort -k1,1 -k2,2n  \
    > /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.sig_da.bed
    

In [19]:
!wc -l /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.sig_da.bed

33323 /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.sig_da.bed


In [20]:
!head /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.sig_da.bed

chr1	4600559	4601117	chr1_4600559_4601117	2.16206789078558	.
chr1	4671564	4672001	chr1_4671564_4672001	-1.30409349798851	.
chr1	4888157	4888775	chr1_4888157_4888775	1.20295110927484	.
chr1	4915898	4916201	chr1_4915898_4916201	0.818977985803797	.
chr1	5015194	5015905	chr1_5015194_5015905	1.43184762199044	.
chr1	5019033	5020138	chr1_5019033_5020138	-0.418523393776327	.
chr1	5031597	5031970	chr1_5031597_5031970	1.31568568310573	.
chr1	5082592	5083621	chr1_5082592_5083621	0.516028339604227	.
chr1	6214071	6215608	chr1_6214071_6215608	-0.281673178101854	.
chr1	6382605	6383624	chr1_6382605_6383624	-1.05227287171606	.


In [23]:
!wc -l /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.txt

68376 /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.txt


In [1]:
!head /data/reddylab/Revathy/collabs/atac-seq_Siklenka/results/deseq2/Th17.72hrs.ASTARR.insert_vs_Th0.72hrs.ASTARR.insert.txt

baseMean	log2FoldChange	lfcSE	pvalue	padj
chr4_143288226_143289757	1647.73336636732	3.81726459784635	0.149421823937938	2.26509166736847e-145	1.54875642756319e-140
chr16_25010772_25012159	1485.52387348661	3.48472723761304	0.140449160020006	4.20040679819601e-137	1.43601407413326e-132
chr2_181099140_181100692	1928.35072840258	2.9614912246334	0.12138931489356	2.19773839527376e-132	5.00901209256145e-128
chr17_24922152_24923413	1302.95899109185	4.05652786705213	0.173908025374107	6.24351466900603e-122	1.06725078873322e-117
chr6_88827500_88829063	1481.49179677977	3.53133982990557	0.152986375873328	3.99629865460984e-119	5.46493841017895e-115
chr1_36965264_36966375	1868.2189488516	2.32734236110937	0.102147391471507	1.24159084180454e-115	1.41489623013975e-111
chr12_70890543_70891870	1399.96979228548	3.98211754387464	0.178451046369452	7.65614613393013e-112	7.47841417010676e-108
chr1_93529464_93530987	1365.00075899984	3.19374550045465	0.143893638020533	3.44938808156298e-110	2.94814887596086e-106
ch